goal: train 10 machine learning predictors: one predictor for each function from the ten protein function categories ("DNA, RNA and nucleotide metabolism", "tail", "head and packaging", "other", "lysis", "connector", "transcription regulation", "moron, auxiliary metabolic gene and host takeover", "unknown function", "integration and excision") 

predictor input: protein features; output: labels (0/1) representing whether the protein serves the specific function

dataset: 360,413 seqs in total - 60% for training, 20% for validation, 20% for testing
-use clustering results to avoid spliting protein seqs in the same cluster (maybe use GroupShuffleSplit from sklearn)

features: 1711-dim

dataset:  
2,318,538 seqs in the original dataset  
927,040 seqs after dropping pcat "unknown_no_hit"  
360,413 unique seqs after dropping duplicated seqs  

subset the negatives;  
- for each function, cluster on the positive dataset  
- try threshold from 0.01 to 1  
- [o] change the dataset splitting strategy  
- [o] model param : is_imbalanced  
- add learning curve (could be find from XGBoost)  
- n_estimators: increased to 10k; may need GPU

# cluster

# training script

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
)
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, Dict, Any
import joblib
import os
import json

In [2]:
def get_df(function_name: str):
    ids = pd.read_csv(f"../dataset/pcat/{function_name}.csv")
    features = pd.read_parquet("../dataset/protein_features_unique.pa")
    features["label"] = features["id"].isin(ids["name"]).astype(int)
    features = features.drop(columns=["md5"])
    return features

In [3]:
def prepare_data(
    df: pd.DataFrame,
    feature_cols: list,
    cluster_mapping: Dict[str, str],
    label_col: str = "label",
    id_col: str = "id",
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Prepare data for training by separating features and labels.

    Args:
        df: DataFrame containing features and labels
        feature_cols: List of feature column names
        cluster_mapping: Dictionary mapping sequence IDs to cluster IDs
        label_col: Name of the label column
        id_col: Name of the ID column

    Returns:
        X: Feature matrix
        y: Label array
        ids: Array of sequence IDs
        groups: Array of cluster IDs for each sequence
    """
    X = df[feature_cols].values
    y = df[label_col].values
    ids = df[id_col].values

    # Get cluster IDs for each sequence
    groups = np.array([cluster_mapping.get(str(id_), "unknown") for id_ in ids])

    return X, y, ids, groups

In [4]:
def split_data(
    X: np.ndarray,
    y: np.ndarray,
    ids: np.ndarray,
    groups: np.ndarray,
    test_size: float = 0.2,
    val_size: float = 0.2,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Split data into train, validation, and test sets while keeping related sequences together.

    Args:
        X: Feature matrix
        y: Label array
        ids: Array of sequence IDs
        groups: Array of cluster IDs for each sequence
        test_size: Proportion of data to use for testing
        val_size: Proportion of data to use for validation
    """
    # First split: separate test set
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
    train_val_idx, test_idx = next(gss.split(X, y, groups=groups))

    X_train_val, X_test = X[train_val_idx], X[test_idx]
    y_train_val, y_test = y[train_val_idx], y[test_idx]
    groups_train_val = groups[train_val_idx]

    # Second split: separate validation set from training set
    val_size_adjusted = val_size / (
        1 - test_size
    )  # Adjust val_size to account for test set
    gss = GroupShuffleSplit(n_splits=1, test_size=val_size_adjusted, random_state=42)
    train_idx, val_idx = next(
        gss.split(X_train_val, y_train_val, groups=groups_train_val)
    )

    X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
    y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]

    return X_train, X_val, X_test, y_train, y_val, y_test


# first split: test 20%
# train:val = 8:2

In [5]:
def train_model(
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_val: np.ndarray,
    y_val: np.ndarray,
    model_params: Dict[str, Any] = None,
) -> Tuple[Any, StandardScaler]:
    """Train an XGBoost classifier with optional hyperparameters."""
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Set default parameters if none provided
    if model_params is None:
        model_params = {
            "n_estimators": 200,
            "max_depth": 6,
            "learning_rate": 0.1,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "min_child_weight": 1,
            "scale_pos_weight": 1,
            "random_state": 42,
            "eval_metric": ["auc", "aucpr"],  # Move eval_metric here
        }

    # Calculate scale_pos_weight based on class imbalance
    n_neg = np.sum(y_train == 0)
    n_pos = np.sum(y_train == 1)
    model_params["scale_pos_weight"] = n_neg / n_pos

    # Train model
    model = XGBClassifier(**model_params)
    model.fit(
        X_train_scaled,
        y_train,
        eval_set=[(X_train_scaled, y_train), (X_val_scaled, y_val)],
        verbose=False,
    )

    return model, scaler

In [6]:
def evaluate_model(
    model: Any,
    scaler: StandardScaler,
    X: np.ndarray,
    y: np.ndarray,
    set_name: str = "",
    threshold: float = 0.5,
) -> Dict[str, float]:
    """Evaluate model performance on a dataset."""

    # no need for scaling!

    X_scaled = scaler.transform(X)
    y_pred_proba = model.predict_proba(X_scaled)[:, 1]
    y_pred = (y_pred_proba >= threshold).astype(int)

    # Calculate MCC
    def matthews(y_true, y_pred):
        from math import sqrt

        """
            P  = Total number of positives
            N  = Total number of negatives
            Tp = number of true positives
            Fp = number of false positives
        """
        if type(y_true) == pd.Series:
            y_true = y_true.values

        P = len([x for x in y_true if x == 1])
        N = len([x for x in y_true if x == 0])

        Tp, Fp = 0, 0
        for i in range(len(y_true)):
            if y_true[i] == 1 and y_pred[i] == 1:
                Tp += 1
            elif y_true[i] == 0 and y_pred[i] == 1:
                Fp += 1

        Tn = N - Fp
        Fn = P - Tp

        try:
            mcc = (Tp * Tn - Fp * Fn) / sqrt(
                (Tn + Fn) * (Tn + Fp) * (Tp + Fn) * (Tp + Fp)
            )
        except ZeroDivisionError:
            mcc = 0
        return (mcc, f"P: {P:_} Tp: {Tp:_} Fp: {Fp:_} N: {N:_} Tn: {Tn:_} Fn: {Fn:_}")

    # Get MCC and confusion matrix values
    mcc, confusion_str = matthews(y, y_pred)

    metrics = {
        f"{set_name}_accuracy": accuracy_score(y, y_pred),
        f"{set_name}_precision": precision_score(y, y_pred),
        f"{set_name}_recall": recall_score(y, y_pred),
        f"{set_name}_f1": f1_score(y, y_pred),
        f"{set_name}_roc_auc": roc_auc_score(y, y_pred_proba),
        f"{set_name}_mcc": mcc,
    }

    # Print metrics
    print(f"\nMetrics for {set_name} (threshold={threshold}):")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

    # Print confusion matrix values
    print(f"\nConfusion Matrix Values:")
    print(confusion_str)

    return metrics

In [7]:
# def plot_feature_importance(model: Any, feature_cols: list, top_n: int = 20):
#     """Plot feature importance from XGBoost model."""
#     importance_scores = model.feature_importances_
#     indices = np.argsort(importance_scores)[::-1]

#     plt.figure(figsize=(12, 6))
#     plt.title("Feature Importances")
#     plt.bar(range(top_n), importance_scores[indices[:top_n]])
#     plt.xticks(range(top_n), [feature_cols[i] for i in indices[:top_n]], rotation=90)
#     plt.tight_layout()
#     plt.show()

#     # Print top N most important features
#     print(f"\nTop {top_n} most important features:")
#     for i in range(top_n):
#         print(f"{feature_cols[indices[i]]}: {importance_scores[indices[i]]:.4f}")

In [8]:
def save_model(
    model: Any, scaler: StandardScaler, feature_cols: list, function_name: str
):
    """Save the trained model and scaler."""
    # Create models directory if it doesn't exist
    os.makedirs("../models", exist_ok=True)

    # Create a dictionary containing all necessary components
    model_data = {"model": model, "scaler": scaler, "feature_cols": feature_cols}

    # Save the model data
    joblib.dump(model_data, f"../models/{function_name}_predictor.joblib")

In [9]:
def train_function_predictor(function_name: str, model_params: Dict[str, Any] = None):
    """Main training pipeline for a specific protein function."""
    # Load cluster mapping
    with open("../dataset/protein_cluster_mapping.json", "r") as f:
        cluster_mapping = json.load(f)

    # get df using function name
    df = get_df(function_name)

    # Get feature columns (excluding 'id' and 'label')
    feature_cols = [col for col in df.columns if col not in ["id", "label"]]

    # Prepare data
    X, y, ids, groups = prepare_data(df, feature_cols, cluster_mapping)

    # Split data
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y, ids, groups)

    # Train model
    model, scaler = train_model(X_train, y_train, X_val, y_val, model_params)

    # Print feature importance (without plot)
    importance_scores = model.feature_importances_
    indices = np.argsort(importance_scores)[::-1]
    print("\nTop 20 most important features:")
    for i in range(20):
        print(f"{feature_cols[indices[i]]}: {importance_scores[indices[i]]:.4f}")

    # Evaluate model on all sets
    print("\n=== Training Set ===")
    train_metrics = evaluate_model(model, scaler, X_train, y_train, "train")

    print("\n=== Validation Set ===")
    val_metrics = evaluate_model(model, scaler, X_val, y_val, "val")

    print("\n=== Test Set ===")
    # Calculate metrics for all thresholds
    thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]  # try 0.01 - 1
    for threshold in thresholds:
        test_metrics = evaluate_model(model, scaler, X_test, y_test, "test", threshold)

    # Save model
    save_model(model, scaler, feature_cols, function_name)

# execution

In [ ]:
model_params = {
    "n_estimators": 10000,
    "max_depth": 8,
    "learning_rate": 0.05,
    "subsample": 0.7,
    "colsample_bytree": 0.7,
    "min_child_weight": 2,
    "random_state": 42,
}

train_function_predictor("tail", model_params)